# A05. Rosters
Sources: 
- MLB Stats API

##### 1. Batting Orders

In [45]:
# # Creates box score variables (consider consolidating with Stats API, it might be possible to read this in once instead of twice)
# def create_box(gamePk):
#     # Read in boxscore as json
#     box = pd.json_normalize(statsapi.boxscore_data(gamePk, timecode=None), record_path='gameBoxInfo')
#     # Take value where label = variable of interest
#     try:
#         weather = box.loc[box['label']=="Weather", "value"].item() 
#     except:
#         weather = "75 degrees, Clear."
#     try:
#         wind = box.loc[box['label']=="Wind", "value"].item()
#     except:
#         wind = "0 mph, L To R."

#     # Create dataframe
#     df = pd.DataFrame({'gamePK': [gamePk], 'weather': [weather], 'wind': [wind]})
        
#     return df

In [18]:
# Creates dataframe of players and their spot in the batting order
def order(gamePk, teamId, date, team="away"):
    list_of_lists = []
    players = statsapi.get("game", {"gamePk": gamePk})['liveData']['boxscore']['teams'][team]['players']
    for player in players:
        id = players[player]['person']['id']
        fullName = players[player]['person']['fullName']
        position = players[player]['position']['name']
        status = players[player]['status']['description']
        try:
            order = statsapi.get("game", {"gamePk": gamePk})['liveData']['boxscore']['teams'][team]['players'][player]['battingOrder']
        except:
            order = np.nan
    
        return_list = [id, fullName, position, status, order]
        list_of_lists.append(return_list)
    
    # Create dataframe
    df = pd.DataFrame(list_of_lists, columns=['id', 'fullName', 'position', 'status', 'order'])
        
    # Game ID
    df['gamePk'] = gamePk
    # Date
    df['date'] = date
    # Team ID #
    df['teamId'] = teamId
        
    return df

##### 2. Rosters

In [1]:
# Creates a roster
def roster(teamId, date):
    # Reformat date to fit function
    date_dash = f"{date[:4]}-{date[4:6]}-{date[6:8]}"

    roster = statsapi.get("team_roster", {"teamId": teamId, "rosterType": "40Man", "date": date_dash, "hydrate": "person"})['roster']

    # Initialize empty lists to store the extracted values
    id_list = []
    full_name_list = []
    first_name_list = []
    last_name_list = []
    position_list = []
    bat_side_list = []
    pitch_hand_list = []

    # Iterate over the roster data
    for player in roster:
        # Extract the values using .get() method and fill missing values with "Missing"
        id_list.append(player['person'].get('id', 'Missing'))
        full_name_list.append(player['person'].get('fullName', 'Missing'))
        first_name_list.append(player['person'].get('firstName', 'Missing'))
        last_name_list.append(player['person'].get('lastName', 'Missing'))
        position_list.append(player['position'].get('name', 'Missing'))
        bat_side_list.append(player['person'].get('batSide', {}).get('description', 'Missing'))
        pitch_hand_list.append(player['person'].get('pitchHand', {}).get('description', 'Missing'))

    # Create the dataframe
    df = pd.DataFrame({
        'id': id_list,
        'fullName': full_name_list,
        'firstName': first_name_list,
        'lastName': last_name_list,
        'position': position_list,
        'batSide': bat_side_list,
        'pitchHand': pitch_hand_list
    })

    date = date.replace("-", "")
    
    # Date
    df['date'] = date
    # Team ID #
    df['teamId'] = teamId
    
    return df